In [2]:
"""
**Une fonction où en entrée nous avons une adresse et en sortie deux points géo (lat/lon).
**Une fonction où en entrée nous avons deux points géo (lat/lon) et en sortie, plusieurs données : 
- Temps d'ensoleillement annuel
- GHI
- Calcul du potentiel de production de kWh / m2 // 15° angle 
- Estimation de la production suffisante à combien de foyers
"""

"\n**Une fonction où en entrée nous avons une adresse et en sortie deux points géo (lat/lon).\n**Une fonction où en entrée nous avons deux points géo (lat/lon) et en sortie, plusieurs données : \n- Temps d'ensoleillement annuel\n- GHI\n- Calcul du potentiel de production de kWh / m2 // 15° angle \n- Estimation de la production suffisante à combien de foyers\n"

In [3]:
#pip install openmeteo-requests
#pip install requests-cache retry-requests numpy pandas

# Import des librairies
import requests
import re

import openmeteo_requests

import pandas as pd
import numpy as np
import requests_cache
from retry_requests import retry


In [4]:
def adress_to_coords(adress : str):
    url = f"https://api-adresse.data.gouv.fr/search/?q={adress.replace(' ', '+')}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            lon, lat = data['features'][0]['geometry']['coordinates']
            return (lat, lon)
    return None

In [45]:
def solar_info(lat, lon):
    """
    # Conversion de l'adresse en points géo (lat/lon)

    url1 = f"https://api-adresse.data.gouv.fr/search/?q={adress.replace(' ', '+')}"
    response1 = requests.get(url1)
    data = response1.json()
    if data['features']:
        lon, lat = data['features'][0]['geometry']['coordinates']
    """
    

    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Paramètres à renseigner
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": f"2024-01-01",
        "end_date": f"2024-12-31",
        "daily": ["sunshine_duration", "temperature_2m_mean"],
        "hourly": "shortwave_radiation",
        "timezone": "Europe/Berlin"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    

    # Process hourly data
    hourly = response.Hourly()
    hourly_shortwave_radiation = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

    hourly_dataframe = pd.DataFrame(data = hourly_data)


    # Nettoyage pour exploitation des données du df Hourly
    hourly_dataframe['month'] = hourly_dataframe['date'].dt.month_name(locale='English')
    hourly_dataframe['month_num'] = hourly_dataframe['date'].dt.month

    hourly_dataframe = hourly_dataframe.iloc[2:]

    df_ghi = hourly_dataframe.groupby(['month', 'month_num']).agg(
        ghi_mean_W_m2=('shortwave_radiation', 'mean')
    )

    df_ghi = df_ghi.sort_values(by='month_num')


    # Process daily data
    daily = response.Daily()
    daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

    daily_dataframe = pd.DataFrame(data = daily_data)


    # Nettoyage pour exploitation des données du df daily
    daily_dataframe['sunshine_duration_hours'] = daily_dataframe['sunshine_duration'] / 3600
    daily_dataframe.drop(columns='sunshine_duration', inplace=True)

    daily_dataframe = daily_dataframe.iloc[1:]

    daily_dataframe['month'] = daily_dataframe['date'].dt.month_name(locale='English')
    daily_dataframe['month_num'] = daily_dataframe['date'].dt.month

    df_weather = daily_dataframe.groupby(['month', 'month_num']).agg(
        temp_mean_c=('temperature_2m_mean', 'mean'),
        sunshine_duration_mean_hour=('sunshine_duration_hours', 'mean')
    )

    df_weather = df_weather.sort_values(by='month_num')

    # Concaténation des dfs

    df = pd.merge(df_weather, df_ghi, on='month')
    
    #===============#
    #    Formules   #
    #===============#

    # Calcul du ghi_journalier
    df['ghi_journalier'] = (df['sunshine_duration_mean_hour'] * df['ghi_mean_W_m2']) / 1000

    # Inclinaison du panneau, standard à 15°, servant de toit sur les portions de pistes cyclables lors de mauvais temps
    df['inclinaison'] = np.sin(np.radians(15))

    # Correction de la température, car rendement baisse au dela de 25°c
    df['correction_temp'] = 1 - (df['temp_mean_c'] - 25) * 0.004

    # Formule pour production energie / jours 
    df['energie_jour_kWh/j/m2'] = round(df['ghi_journalier'] * df['inclinaison'] * df['correction_temp'],2)

    # Multiplication pour production energie / mois 
    df['energie_mois_kWh/mois/m2'] = round(df['energie_jour_kWh/j/m2'] * 30.41,2)

    # Création d'un df de simulation
    df_copy = df.copy()

    df_copy.drop(columns=['temp_mean_c', 'sunshine_duration_mean_hour', 'ghi_mean_W_m2', 'ghi_journalier', 'inclinaison', 'correction_temp'], inplace=True)

    df_copy['prod_1m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2']
    df_copy['prod_5m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2'] * 5
    df_copy['prod_10m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2'] * 10
    df_copy['prod_20m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2'] * 20


    # Concaténation des dfs

    df_final = pd.merge(df, df_copy, on='month')

    df_final.drop(columns=['energie_jour_kWh/j/m2_x', 'energie_mois_kWh/mois/m2_x'], inplace=True)
    df_final.rename(columns={
        'energie_jour_kWh/j/m2_y':'energie_jour_kWh/j/m2',
        'energie_mois_kWh/mois/m2_y' : 'energie_mois_kWh/mois/m2'
    }, inplace=True)


    # Calcul production annuelle 
    # Calcul de la production annuelle pour chaque surface
    
    df_annuel = pd.DataFrame({
        "Surface du panneau (m2)": [1],
        "Énergie produite annuelle (kWh)":
            round(df_final['energie_mois_kWh/mois/m2'].sum(), 2),
        
    })



    """
    print(f"Données concernant ces coordonnées : ")
    print(f"Latitude : {lat}")
    print(f"Latitude : {lon}")
    print(f"Données concernant cette année : ")
    print(f"2024")
    """
    return df_annuel
    #return df_final
    

In [46]:
solar_info('1.09673702748082','43.62109781579031')

,Surface du panneau (m2),Énergie produite annuelle (kWh)
0,1,246.32


In [21]:
df, annuel = solar_info(45.9, 6.1)
print(annuel)


   Surface (m2)  Énergie annuelle (kWh)
0             1                  148.39
1             5                  741.95
2            10                 1483.90
3            20                 2967.80


**Avec une production de 642 kWh/mois pour 20 m² de panneaux posés sur des ombrières en bord de voie verte, tu obtiens une énergie relativement conséquente (≈ 21 kWh/jour)**

**🚲 1. Recharge de vélos électriques ou trottinettes**

***✅ Idéal pour : stations de recharge en libre-service***
🔋 Une batterie de VAE (vélo à assistance électrique) consomme ~0,5 kWh par charge.

👉 Tu pourrais recharger ~42 vélos/jour (21 ÷ 0,5).

💡 Intéressant pour :

Loueurs de vélos en libre-service,

Cyclotouristes de passage,

Usagers réguliers de la voie verte.

**🧑‍🤝‍🧑 2. Alimentation d’infrastructures locales / urbaines**

***✅ Exemple d’usages :***
Éclairage LED public de la voie verte (faible conso) : ~30 à 80 W par lampadaire

21 kWh permettent d’alimenter ~40 à 60 lampadaires LED 8 h/jour

Caméras de sécurité ou capteurs météo/CO2

Bornes d’appel d’urgence ou bornes Wi-Fi

**🔌 3. Points de recharge pour véhicules légers**

***✅ Exemple :***
⚡️ Recharge de petits véhicules électriques type golfette, scooter, triporteur

✅ Pratique pour collectivités ou services techniques qui entretiennent la voie verte

**🏕️ 4. Services aux usagers : micro-stations solaires**

***Tu pourrais créer des stations solaires multi-usages :***

Bornes de recharge téléphones/ordinateurs

Micro-pompes à eau potable ou fontaines filtrantes

Zones de pause ombragées avec prises USB

Écran d’info solaire (météo, sentier, biodiversité...)

**⚡ 5. Injection au réseau (revente)**

***L’énergie peut être injectée dans le réseau public via Enedis, avec contrat d’achat (ex : EDF OA solaire).***

Environ 0,10 à 0,13 €/kWh injecté, soit ~65 €/mois pour 642 kWh injectés.

📉 Moins impactant socialement mais valorisable économiquement.

**♻️ 6. Stockage local (batteries) pour usages différés**

***Stocker l’énergie solaire pour l’utiliser la nuit :***

Éclairage nocturne,

Stations rechargeables ouvertes 24h/24,

Autonomie en cas de coupure de courant.


